# Support Vector Machines (SVM) - Linear Kernel

In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd

In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [4]:
from joblib import Parallel, delayed
def process(vectorizer):
    evaluation = list()
    usernames = list(np.unique(data["username"]))
    
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(   
                data, author1, author2)

            clf = AuthorClassifier(vectorizer, clf=AdaBoostClassifier(random_state=42), scaler=MaxAbsScaler())
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            evaluation.append(clf.evaluate(y_test, y_pred))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
    
    return (vectorizer_str, metrics)
    
results = Parallel(n_jobs=6)(delayed(process)(vectorizer) for vectorizer in vectorizers)
print(results) 

Running => CountVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(4, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(3, 8))
Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f06472b3c10>>)
Running => CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f24fa339c70>>)
Running => TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7efc6f30ac70>>)
Running => TfidfVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f06472b3c40>>)
Running => TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer=

In [5]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/ada_boost.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.847452,0.848372,0.851282,0.847862,0.917828,CountVectorizer(tokenizer=<bound method TweetT...
1,0.849390,0.850168,0.853319,0.849841,0.919154,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.851744,0.852260,0.853313,0.851932,0.922067,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.771047,0.773888,0.781773,0.773208,0.851739,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.791806,0.793201,0.796886,0.792671,0.874451,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.838390,0.838961,0.841046,0.838733,0.907580,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.833414,0.834361,0.837676,0.833976,0.903398,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.840330,0.840587,0.841323,0.840529,0.912846,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.765336,0.767222,0.772302,0.767026,0.842941,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.780874,0.781519,0.783757,0.781523,0.860839,"TfidfVectorizer(analyzer='char', ngram_range=(..."


Row with max sum of columns (remember that by the result of f1-score we use the precision and recall score)

In [6]:
print(metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()])
print("\nVectorizer:", metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()]["vectorizer"])

f1_macro                                                    0.851744
recall_macro                                                 0.85226
precision_macro                                             0.853313
accuracy                                                    0.851932
auc_score                                                   0.922067
vectorizer         CountVectorizer(analyzer='char', ngram_range=(...
Name: 2, dtype: object

Vectorizer: CountVectorizer(analyzer='char', ngram_range=(1, 5))


In [7]:
pd.read_csv("../../results/ada_boost.csv").drop("Unnamed: 0", axis=1)

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.847452,0.848372,0.851282,0.847862,0.917828,CountVectorizer(tokenizer=<bound method TweetT...
1,0.849390,0.850168,0.853319,0.849841,0.919154,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.851744,0.852260,0.853313,0.851932,0.922067,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.771047,0.773888,0.781773,0.773208,0.851739,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.791806,0.793201,0.796886,0.792671,0.874451,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.838390,0.838961,0.841046,0.838733,0.907580,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.833414,0.834361,0.837676,0.833976,0.903398,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.840330,0.840587,0.841323,0.840529,0.912846,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.765336,0.767222,0.772302,0.767026,0.842941,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.780874,0.781519,0.783757,0.781523,0.860839,"TfidfVectorizer(analyzer='char', ngram_range=(..."
